In [3]:
# get the code
import sys
import os
sys.path.append('../code')

# set the paths
home = '/Users/martinscience'
home = '/Users/mahtin'


somvar_path = os.path.join(home, "Dropbox/Icke/Work/somVar")
testdata = os.path.join(somvar_path, "testdata")
ebdata = os.path.join(somvar_path, "tooldata/EBdata")
pon_path = os.path.join(testdata, "PON")

static = os.path.join(home, "Dropbox/Icke/Work/static")

## create EBmatrix from different input
+ filterVar.mawk is the input fork

In [4]:
from ebraw import tumor2matrix

EBconfig = {
    "temp_dir": os.path.join(ebdata, "temp"),
    "pon_path": pon_path,
    "mawk_path": "../shell",
    "genome_split": os.path.join(static, "genome/gatk/hg38/split"),
    "MAPQ": 20,
    "Q": 25,
    "fit_pen": 0.5,
    "count_dict": {0:"alt+", 1:"alt-", 2:"depth+", 3:"depth-"},
    "use_cache": True
}


bam_file = f"{testdata}/bam/002_A.chr7.bam"
pileup_file = os.path.join(ebdata, "pileup/002_A-B.pileup.gz")
mut_file = os.path.join(ebdata, "table/002_A-B.csv")
pon_list = "PON_chr7.txt"
chrom = "chr7"

# df = bam2matrix(mut_file, pileup=pileup_file, chrom=chrom, pon_list=pon_list, EBconfig=EBconfig)
matrix_df = tumor2matrix(mut_file, bam=bam_file, chrom=chrom, pon_list=pon_list, EBconfig=EBconfig)
matrix_df

08:05:25 : 02_B.chr7.bam from PON list matches /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/bam/002_A.chr7.bam and is removed from pon_list. If not desired, change names
$ samtools mpileup -Q 25 -q 20 -l /Users/mahtin/Dropbox/Icke/Work/somVar/tooldata/EBdata/temp/002_A-B_chr7.bed -f /Users/mahtin/Dropbox/Icke/Work/static/genome/gatk/hg38/split/chr7.fa -r chr7 /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/bam/002_A.chr7.bam | ../shell/cleanpileup.mawk | ../shell/pile2count.mawk | ../shell/tumor2matrix.mawk /Users/mahtin/Dropbox/Icke/Work/somVar/tooldata/EBdata/table/002_A-B.csv chr7 -P /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/matrix/chr7.pon -x 2


,Chr,Start,End,Ref,Alt,Tumor,PON+,PON-
0,chr7,35326,35326,C,T,0-1=0-1,0|0|0|0=4|6|31|0,0|0|0|3=2|0|0|3
1,chr7,248451,248451,G,A,0-0=6-30,1|0|1|0=69|33|86|65,0|1|0|0=102|60|135|75
2,chr7,248458,248458,T,C,0-0=6-28,1|0|1|0=49|20|66|49,0|0|0|0=114|60|139|79
3,chr7,248482,248482,-,A,0-0=2-21,0|0|0|0=23|8|25|22,0|1|1|0=95|51|130|68
4,chr7,248487,248487,G,A,0-0=1-21,0|0|1|0=11|6|16|18,0|1|2|0=86|52|121|66
...,...,...,...,...,...,...,...,...
7010,chr7,159142337,159142338,T,-,1-0=8-0,1|1|1|1=52|59|71|83,0|0|0|0=4|2|1|2
7011,chr7,159144277,159144277,C,A,0-0=25-3,0|0|0|0=166|241|218|113,0|0|0|0=27|33|27|17
7012,chr7,159144683,159144683,C,T,5-3=5-9,0|0|0|0=100|120|120|85,0|0|0|0=77|156|138|43
7013,chr7,159144893,159144893,T,C,0-1=1-8,0|22|5|6=7|22|5|8,0|65|54|12=30|65|54|28


### save matrix file

In [ ]:
matrix_df.to_csv(os.path.join(ebdata, "matrix/02_A.chr7.matrix"), sep="\t", index=False)

## matrix2AB

In [ ]:
from matrix2AB import matrix2AB_multi

AB_config = {
    "fit_pen": 0.5,
    "threads": 8,
    "chunk_size":500,
    "zero_path": os.path.join(pon_path, "zero")
}
AB_df = matrix2AB_multi(matrix_df, AB_config)
AB_df

### save AB file

In [ ]:
AB_df.to_csv(os.path.join(ebdata, "AB/02_A.chr7.AB"), sep="\t", index=False)